In [1]:
pwd

'C:\\Users\\laad_'

### 1. Install and Import Baseline Dependencies

In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [6]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests
import re
from transformers import pipeline
import csv

### 2. Setup Model

In [8]:
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

### 3. Summarize a Single Article

In [10]:
url = "https://www.thestreet.com/technology/toyota-about-to-make-a-big-change-to-challenge-tesla?puc=yahoo&cm_ven=YAHOO"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
paragraphs =  soup.find_all('p')

In [11]:
paragraphs

[<p>Toyota <span> (<strong><a href="https://www.thestreet.com/quote/TM">TM</a></strong>) </span> seems set to do an about-face. </p>,
 <p>In the face of the industrywide race toward electric vehicles, the Japanese automaker has boasted of its unique strategy.</p>,
 <p>For Toyota, the world's second largest automotive group based on market value behind Tesla <span> (<strong><a href="https://www.thestreet.com/quote/TSLA">TSLA</a></strong>) </span>, a strategy combining gasoline vehicles and less-polluting ones is the recipe. </p>,
 <p>Instead of going full electric, the group has long favored hybrids, which have earned it consistent support from environmental activists. The Toyota Prius has been a model for environmentally conscious consumers.</p>,
 <p>Toyota has tried to go it alone on one strategy: <a href="https://www.thestreet.com/investing/etfs/how-to-invest-in-hydrogen-an-alternative-to-fossil-fuel-energy">hydrogen</a>-fuel-cell cars. The Toyota Mirai is the only fuel-cell car on t

In [15]:
text = [paragraph.text for paragraph in paragraphs]
words = ' '.join(text).split(' ')[:400]
ARTICLE = ' '.join(words)

In [17]:
words

['Toyota\xa0',
 '(TM)',
 '',
 'seems',
 'set',
 'to',
 'do',
 'an',
 'about-face.\xa0',
 'In',
 'the',
 'face',
 'of',
 'the',
 'industrywide',
 'race',
 'toward',
 'electric',
 'vehicles,',
 'the',
 'Japanese',
 'automaker',
 'has',
 'boasted',
 'of',
 'its',
 'unique',
 'strategy.',
 'For',
 'Toyota,',
 'the',
 "world's",
 'second',
 'largest',
 'automotive',
 'group',
 'based',
 'on',
 'market',
 'value',
 'behind',
 'Tesla\xa0',
 '(TSLA)',
 ',',
 'a',
 'strategy',
 'combining',
 'gasoline',
 'vehicles',
 'and',
 'less-polluting',
 'ones',
 'is',
 'the',
 'recipe.\xa0',
 'Instead',
 'of',
 'going',
 'full',
 'electric,',
 'the',
 'group',
 'has',
 'long',
 'favored',
 'hybrids,',
 'which',
 'have',
 'earned',
 'it',
 'consistent',
 'support',
 'from',
 'environmental',
 'activists.',
 'The',
 'Toyota',
 'Prius',
 'has',
 'been',
 'a',
 'model',
 'for',
 'environmentally',
 'conscious',
 'consumers.',
 'Toyota',
 'has',
 'tried',
 'to',
 'go',
 'it',
 'alone',
 'on',
 'one',
 'strate

In [19]:
len(words)

400

In [21]:
ARTICLE

'Toyota\xa0 (TM)  seems set to do an about-face.\xa0 In the face of the industrywide race toward electric vehicles, the Japanese automaker has boasted of its unique strategy. For Toyota, the world\'s second largest automotive group based on market value behind Tesla\xa0 (TSLA) , a strategy combining gasoline vehicles and less-polluting ones is the recipe.\xa0 Instead of going full electric, the group has long favored hybrids, which have earned it consistent support from environmental activists. The Toyota Prius has been a model for environmentally conscious consumers. Toyota has tried to go it alone on one strategy: hydrogen-fuel-cell cars. The Toyota Mirai is the only fuel-cell car on the market.\xa0 In early October, Chief Executive Akio Toyoda confirmed that the vehicle manufacturer would unveil a lineup of electrified vehicles that run on liquid hydrogen and traditional rechargeable lithium-ion battery packs. Toyota hopes to invest $70 billion in electrified vehicles, with half tha

In [43]:
input_ids = tokenizer.encode(ARTICLE, return_tensors='pt')
output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
summary = tokenizer.decode(output[0], skip_special_tokens=True)

In [42]:
summary

'Work on existing electric vehicle platforms has been halted, Reuters reports.'

In [40]:
input_ids

tensor([[ 6190,   143,  2259,   158,  1045,   323,   112,   171,   142,   160,
           121,  7998,   107,   222,   109,   749,   113,   109,   503,  5650,
          1580,  2153,  2425,  2070,   108,   109,  2769, 32930,   148, 39630,
           113,   203,   620,  1520,   107,   321,  6190,   108,   109,   278,
           131,   116,   453,  1368,  6090,   456,   451,   124,   407,   564,
           893, 11997,   143, 72495,   251,   158,   110,   108,   114,  1520,
          6839, 14255,  2070,   111,   478,   121, 32952, 34752,  1304,   117,
           109,  1869,   107,  3054,   113,   313,   357,  2425,   108,   109,
           456,   148,   300, 17902, 34377,   108,   162,   133,  3032,   126,
          3302,   337,   135,  2249, 10162,   107,   139,  6190, 37269,   148,
           174,   114,   861,   118,  8641,  7394,  2359,   107,  6190,   148,
          1424,   112,   275,   126,  1600,   124,   156,  1520,   151, 13535,
           121, 25475,   121, 12052,  1732,   107,  

### 4. Building a News and Sentiment Pipeline

In [54]:
monitored_tickers = ['ETH', 'TSLA', 'BTC']

#### 4.1. Search for Stock News using Google and Yahoo Finance

In [55]:
def search_for_stock_news_urls(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&tbm=nws".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs 

In [59]:
#search_for_stock_news_urls('ETH')

In [60]:
raw_urls = {ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

{'ETH': ['/?sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQOwgC',
  '/search?q=yahoo+finance+ETH&tbm=nws&ie=UTF-8&gbv=1&sei=JdhYY6mJFYHc5OUPqtGukAE',
  '/search?q=yahoo+finance+ETH&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQ_AUIBSgA',
  '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQ_AUIBygC',
  '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQ_AUICCgD',
  'https://maps.google.com/maps?q=yahoo+finance+ETH&um=1&ie=UTF-8&sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQ_AUICSgE',
  '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQ_AUICigF',
  '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQ_AUICygG',
  '/advanced_search',
  '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=nws&source=lnt&tbs=qdr:h&sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQpwUIDQ',
  '/search?q=yahoo+finance+ETH&ie=U

In [62]:
#raw_urls['ETH']

['/?sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQOwgC',
 '/search?q=yahoo+finance+ETH&tbm=nws&ie=UTF-8&gbv=1&sei=JdhYY6mJFYHc5OUPqtGukAE',
 '/search?q=yahoo+finance+ETH&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQ_AUIBSgA',
 '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQ_AUIBygC',
 '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQ_AUICCgD',
 'https://maps.google.com/maps?q=yahoo+finance+ETH&um=1&ie=UTF-8&sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQ_AUICSgE',
 '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQ_AUICigF',
 '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQ_AUICygG',
 '/advanced_search',
 '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=nws&source=lnt&tbs=qdr:h&sa=X&ved=0ahUKEwjpr__Apv36AhUBLrkGHaqoCxIQpwUIDQ',
 '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=nws&sourc

#### 4.2. Strip out unwanted URLs
 

In [64]:
import re

In [65]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [66]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls: 
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [68]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_tickers}
cleaned_urls

{'ETH': ['https://finance.yahoo.com/news/ethereum-merge-vaults-cryptocurrency-past-020111129.html',
  'https://finance.yahoo.com/news/censorship-fork-ethereum-111500357.html',
  'https://finance.yahoo.com/news/defi-options-platform-opyns-crab-093839942.html',
  'https://uk.finance.yahoo.com/news/bitcoin-price-crypto-ethereum-binance-094851573.html',
  'https://finance.yahoo.com/news/reversibility-ethereum-benefits-pitfalls-190719333.html',
  'https://finance.yahoo.com/news/celsius-once-solicited-donations-ukraine-073839535.html',
  'https://uk.finance.yahoo.com/news/ethereum-price-shanghai-upgrade-090725986.html',
  'https://uk.finance.yahoo.com/news/rishi-sunak-crypto-hub-bitcoin-123727766.html',
  'https://uk.finance.yahoo.com/news/bitcoin-ethereum-price-latest-crypto-market-093018342.html',
  'https://finance.yahoo.com/news/interview-cronos-provides-first-ethereum-130310177.html'],
 'TSLA': ['https://www.reuters.com/business/autos-transportation/exclusive-toyota-scrambles-ev-reboot-

#### 4.3. Search and Scrape Cleaned URLs
 

In [69]:
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs: 
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [72]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'ETH': ['Ethereum won plaudits and the spotlight two weeks ago for smoothly pushing through its much-hyped Merge, a historic shift to a different “proof-of-stake” blockchain system designed to drastically reduce energy consumption – roughly 99% by some estimates. Now, the second-biggest blockchain appears to be proving itself on another promise of the Merge: greater inflation-resistance, a characteristic that’s usually more closely associated with Ethereum’s bigger and better-known rival, Bitcoin. In the days since the Merge, the annualized net issuance rate of Ethereum’s native cryptocurrency, ether (ETH), has fallen to a range of 0% to 0.7%, estimates Lucas Outumuro, head of research at crypto data and analysis firm IntoTheBlock. That compares with about 3.5% prior to the Merge. The net issuance rate, also referred to as the inflation rate, is essentially the new supply divided by the existing supply. The website Ultra Sound Money puts the annualized inflation rate at 0.19%, based o

In [73]:
articles['ETH'][2]

'A decentralized finance (DeFi) product that launched earlier this year to help investors profit from a sideways trending market is generating desired results. The "Crab" strategy, designed by leading DeFi options platform Opyn, is built to make money during bouts of low ether (ETH) price volatility. It has earned 14% returns in U.S. dollar terms and 42% in ether terms since its v2 went live in late July. Ether has dropped 20% since the end of July, however, the cryptocurrency has primarily traded between $1,100 to $1,300 in the past four weeks. The strategy is yet another example of the DeFi industry helping investors bypass complexities involved with traditional option strategies like "short straddle" often set up by sophisticated market participants during a market lull. In case of a short straddle, a trader sells bullish and bearish options contracts called call and put on a centralized exchange. If the market stays rangebound, the short straddle makes money. Setting up such multi-

#### 4.4. Summarise all Articles

In [75]:
def summarize(articles):
    summaries = []
    for article in articles:
        input_ids = tokenizer.encode(article, return_tensors='pt')
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [77]:
summaries = {ticker:summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

{'ETH': ['Ethereum’s net issuance rate has fallen since the Merge. Bitcoin’s inflation rate is currently about 1.75%',
  'Core principles like ‘credible neutrality’ seen at odds with regulators. Two versions of the Chain: one regulated, one not',
  'Ether has dropped 20% since the end of July, but has rebounded in past four weeks. DeFi option platform Opyn says it has seen a 505% increase in users.',
  'Analysts claim a price indicator is pointing to a significant move.',
  'Reversibility would probably be a popular feature, especially among risk-averse. Stanford researchers don’t intend for Ethereum to be 100% reversible',
  'Critics question whether donations went to the right place. Bitcoin and Ether donations flowed to Celsius-controlled wallets',
  'The Ethereum Foundation is working on the next major upgrade. Ethereum Foundation is launching a pre-Shangdong testnet',
  'Rishi Sunak appointed prime minister after Liz Truss resigned. Key crypto-analysts ask what could happen under 

In [79]:
summaries['BTC']

['Consumer prices rose 8.2% in September from a year earlier. Bitcoin climbed back above Futures on the S&P 500 after the release',
 'Cathie Wood says she bought $100,000 worth of bitcoin in 2015. ARK Next Generation Internet Fund holds about 5.9 million shares of GBTC',
 'We are aware of the issue and are working to resolve it.',
 'Analysts claim a price indicator is pointing to a significant move.',
 'Grandparents, Bitcoin (BTC) and Litecoin (LTC) are showing signs of age.',
 'Australia says central bank digital currency will be treated as foreign currency.',
 'Rishi Sunak appointed prime minister after Liz Truss resigned. Key crypto-analysts ask what could happen under his premiership',
 'About six to ten miners in pipeline, CEO says. DeFi loans are being offered on a ‘full recourse’ basis',
 'Sterling falls to record low against dollar on Friday. Bitcoin made a surprise rally as equity markets slide',
 'Peter Brandt expects coin’s price to bottom at $13,000. Says Fed needs to ‘rega

### 5. Adding Sentiment Analysis

In [81]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [83]:
sentiment(summaries['BTC'])

[{'label': 'POSITIVE', 'score': 0.5207523107528687},
 {'label': 'NEGATIVE', 'score': 0.8885747790336609},
 {'label': 'POSITIVE', 'score': 0.9979088306427002},
 {'label': 'NEGATIVE', 'score': 0.5154085755348206},
 {'label': 'NEGATIVE', 'score': 0.7475226521492004},
 {'label': 'NEGATIVE', 'score': 0.9965127110481262},
 {'label': 'NEGATIVE', 'score': 0.9811481833457947},
 {'label': 'NEGATIVE', 'score': 0.9894652366638184},
 {'label': 'NEGATIVE', 'score': 0.8797296285629272},
 {'label': 'NEGATIVE', 'score': 0.9996774196624756}]

In [84]:
scores = {ticker:sentiment(summaries[ticker]) for ticker in monitored_tickers}
scores

{'ETH': [{'label': 'NEGATIVE', 'score': 0.999365508556366},
  {'label': 'NEGATIVE', 'score': 0.8471165299415588},
  {'label': 'POSITIVE', 'score': 0.6509277820587158},
  {'label': 'NEGATIVE', 'score': 0.5154085755348206},
  {'label': 'NEGATIVE', 'score': 0.9586257338523865},
  {'label': 'NEGATIVE', 'score': 0.9961395859718323},
  {'label': 'NEGATIVE', 'score': 0.9860681295394897},
  {'label': 'NEGATIVE', 'score': 0.9811481833457947},
  {'label': 'NEGATIVE', 'score': 0.9978916049003601},
  {'label': 'POSITIVE', 'score': 0.9870172142982483}],
 'TSLA': [{'label': 'NEGATIVE', 'score': 0.9994420409202576},
  {'label': 'POSITIVE', 'score': 0.9971206188201904},
  {'label': 'NEGATIVE', 'score': 0.6654184460639954},
  {'label': 'POSITIVE', 'score': 0.9985582232475281},
  {'label': 'NEGATIVE', 'score': 0.9994505047798157},
  {'label': 'NEGATIVE', 'score': 0.9938549399375916},
  {'label': 'NEGATIVE', 'score': 0.9967318773269653},
  {'label': 'NEGATIVE', 'score': 0.9980599284172058},
  {'label': '

In [86]:
print(summaries['ETH'][3], scores['ETH'][3]['label'], scores['ETH'][3]['score'])

Analysts claim a price indicator is pointing to a significant move. NEGATIVE 0.5154085755348206


In [88]:
scores['BTC'][0]['score']

0.5207523107528687

### 6. Exporting Results to CSV


In [90]:
summaries

{'ETH': ['Ethereum’s net issuance rate has fallen since the Merge. Bitcoin’s inflation rate is currently about 1.75%',
  'Core principles like ‘credible neutrality’ seen at odds with regulators. Two versions of the Chain: one regulated, one not',
  'Ether has dropped 20% since the end of July, but has rebounded in past four weeks. DeFi option platform Opyn says it has seen a 505% increase in users.',
  'Analysts claim a price indicator is pointing to a significant move.',
  'Reversibility would probably be a popular feature, especially among risk-averse. Stanford researchers don’t intend for Ethereum to be 100% reversible',
  'Critics question whether donations went to the right place. Bitcoin and Ether donations flowed to Celsius-controlled wallets',
  'The Ethereum Foundation is working on the next major upgrade. Ethereum Foundation is launching a pre-Shangdong testnet',
  'Rishi Sunak appointed prime minister after Liz Truss resigned. Key crypto-analysts ask what could happen under 

In [91]:
scores

{'ETH': [{'label': 'NEGATIVE', 'score': 0.999365508556366},
  {'label': 'NEGATIVE', 'score': 0.8471165299415588},
  {'label': 'POSITIVE', 'score': 0.6509277820587158},
  {'label': 'NEGATIVE', 'score': 0.5154085755348206},
  {'label': 'NEGATIVE', 'score': 0.9586257338523865},
  {'label': 'NEGATIVE', 'score': 0.9961395859718323},
  {'label': 'NEGATIVE', 'score': 0.9860681295394897},
  {'label': 'NEGATIVE', 'score': 0.9811481833457947},
  {'label': 'NEGATIVE', 'score': 0.9978916049003601},
  {'label': 'POSITIVE', 'score': 0.9870172142982483}],
 'TSLA': [{'label': 'NEGATIVE', 'score': 0.9994420409202576},
  {'label': 'POSITIVE', 'score': 0.9971206188201904},
  {'label': 'NEGATIVE', 'score': 0.6654184460639954},
  {'label': 'POSITIVE', 'score': 0.9985582232475281},
  {'label': 'NEGATIVE', 'score': 0.9994505047798157},
  {'label': 'NEGATIVE', 'score': 0.9938549399375916},
  {'label': 'NEGATIVE', 'score': 0.9967318773269653},
  {'label': 'NEGATIVE', 'score': 0.9980599284172058},
  {'label': '

In [92]:
cleaned_urls

{'ETH': ['https://finance.yahoo.com/news/ethereum-merge-vaults-cryptocurrency-past-020111129.html',
  'https://finance.yahoo.com/news/censorship-fork-ethereum-111500357.html',
  'https://finance.yahoo.com/news/defi-options-platform-opyns-crab-093839942.html',
  'https://uk.finance.yahoo.com/news/bitcoin-price-crypto-ethereum-binance-094851573.html',
  'https://finance.yahoo.com/news/reversibility-ethereum-benefits-pitfalls-190719333.html',
  'https://finance.yahoo.com/news/celsius-once-solicited-donations-ukraine-073839535.html',
  'https://uk.finance.yahoo.com/news/ethereum-price-shanghai-upgrade-090725986.html',
  'https://uk.finance.yahoo.com/news/rishi-sunak-crypto-hub-bitcoin-123727766.html',
  'https://uk.finance.yahoo.com/news/bitcoin-ethereum-price-latest-crypto-market-093018342.html',
  'https://finance.yahoo.com/news/interview-cronos-provides-first-ethereum-130310177.html'],
 'TSLA': ['https://www.reuters.com/business/autos-transportation/exclusive-toyota-scrambles-ev-reboot-

In [93]:
range(len(summaries['ETH']))

range(0, 10)

In [97]:
summaries['TSLA'][8]

'‘A threat of this magnitude is reckless,’ says draft of open letter. Musk must complete his $44 billion acquisition by Friday'

In [99]:
print(summaries['TSLA'][8], scores['TSLA'][8]['label'], scores['TSLA'][8]['score'])

‘A threat of this magnitude is reckless,’ says draft of open letter. Musk must complete his $44 billion acquisition by Friday NEGATIVE 0.9948495030403137


In [100]:
def create_output_array(summaries, scores, urls):
    output = []
    for ticker in monitored_tickers:
        for counter in range(len(summaries[ticker])):
            output_this = [
                ticker,
                summaries[ticker][counter],
                scores[ticker][counter]['label'],
                scores[ticker][counter]['score'],
                urls[ticker][counter]
            ]
            output.append(output_this)
    return output

In [101]:
final_output = create_output_array(summaries, scores, cleaned_urls)
final_output

[['ETH',
  'Ethereum’s net issuance rate has fallen since the Merge. Bitcoin’s inflation rate is currently about 1.75%',
  'NEGATIVE',
  0.999365508556366,
  'https://finance.yahoo.com/news/ethereum-merge-vaults-cryptocurrency-past-020111129.html'],
 ['ETH',
  'Core principles like ‘credible neutrality’ seen at odds with regulators. Two versions of the Chain: one regulated, one not',
  'NEGATIVE',
  0.8471165299415588,
  'https://finance.yahoo.com/news/censorship-fork-ethereum-111500357.html'],
 ['ETH',
  'Ether has dropped 20% since the end of July, but has rebounded in past four weeks. DeFi option platform Opyn says it has seen a 505% increase in users.',
  'POSITIVE',
  0.6509277820587158,
  'https://finance.yahoo.com/news/defi-options-platform-opyns-crab-093839942.html'],
 ['ETH',
  'Analysts claim a price indicator is pointing to a significant move.',
  'NEGATIVE',
  0.5154085755348206,
  'https://uk.finance.yahoo.com/news/bitcoin-price-crypto-ethereum-binance-094851573.html'],
 [

In [102]:
final_output.insert(0, ['Ticker', 'Summary', 'Label', 'Confidence', 'URL'])

In [108]:
final_output

[['Ticker', 'Summary', 'Label', 'Confidence', 'URL'],
 ['ETH',
  'Ethereum’s net issuance rate has fallen since the Merge. Bitcoin’s inflation rate is currently about 1.75%',
  'NEGATIVE',
  0.999365508556366,
  'https://finance.yahoo.com/news/ethereum-merge-vaults-cryptocurrency-past-020111129.html'],
 ['ETH',
  'Core principles like ‘credible neutrality’ seen at odds with regulators. Two versions of the Chain: one regulated, one not',
  'NEGATIVE',
  0.8471165299415588,
  'https://finance.yahoo.com/news/censorship-fork-ethereum-111500357.html'],
 ['ETH',
  'Ether has dropped 20% since the end of July, but has rebounded in past four weeks. DeFi option platform Opyn says it has seen a 505% increase in users.',
  'POSITIVE',
  0.6509277820587158,
  'https://finance.yahoo.com/news/defi-options-platform-opyns-crab-093839942.html'],
 ['ETH',
  'Analysts claim a price indicator is pointing to a significant move.',
  'NEGATIVE',
  0.5154085755348206,
  'https://uk.finance.yahoo.com/news/bitc

In [110]:
import csv
with open('assetsummaries.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerows(final_output)